## Final Pipeline for all drugs using PCA
**Issues**
* Dobutamine and Isoprenaline have Accuracy of 1.00 🤙🏼🤙🏼

### Initialise the dataframes

In [53]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [54]:
PCA_df = pd.read_csv("FeaturesPCA_dropped.csv",index_col=[0])
PCA_df.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,-0.572855,0.775703,-0.555175,-0.668609,0.206971,0.311069,0.004777,-0.218921,0.021354,0.522483,...,0.065758,0.117052,0.313998,0.083378,0.270367,0.613181,-0.032338,-0.291381,-0.346303,-0.057003
1,0.378881,-0.054257,-0.140639,-0.745776,0.054000,-0.078432,0.065719,-0.312667,-0.300151,0.531537,...,0.184845,0.130041,-0.281455,-0.216349,0.080771,0.129352,0.079545,-0.117099,0.032437,0.089642
2,0.378855,-0.298639,-0.214987,-0.136841,-0.174487,0.944021,0.676052,0.005444,0.292429,-0.035979,...,-0.126694,0.162750,0.056158,0.233953,0.037804,-0.182907,0.073337,0.190404,0.210340,-0.395110
3,-0.385298,0.650624,-0.147322,-0.262479,0.156089,0.317733,-0.001556,-0.057074,0.389654,0.725043,...,-0.120545,-0.025797,0.037880,0.050543,0.093189,0.009083,0.240156,-0.030918,0.150497,-0.147013
4,-0.721262,-0.141627,0.296722,0.191071,-0.157643,-0.106280,0.209430,0.399258,-0.148421,0.039294,...,0.078502,-0.083293,0.046128,-0.067033,0.065144,0.102891,0.149844,0.059303,-0.015103,-0.049884


In [55]:
df = pd.read_csv("drug_onehot_latest.csv")
df

,inpatient.number,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
0,722128,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
1,723327,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
2,723617,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,724385,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
4,725509,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998,870258,0,0,0,0,1,1,1,0,0,0,0,1,1,0,1,0,1,0
1999,870646,0,1,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0
2000,879601,1,1,1,1,0,1,1,0,0,0,1,1,1,0,1,0,0,0
2001,905163,0,1,1,1,0,1,0,0,0,0,1,1,0,1,1,1,0,0


In [56]:
df.drop(['inpatient.number'], axis=1,inplace=True)

### Initialise the Neural net and The pipeline

In [57]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(128, input_dim=65, activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=65,verbose = 0)
    else:
        model.fit(X_train, y_train, epochs=50, class_weight = weights)
    
    model.evaluate(X_test, y_test,verbose = 0)
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

In [58]:
def model(q, trainer):

    count_class_0, count_class_1 = trainer[q].value_counts()[0], trainer[q].value_counts()[1]

    # Divide by class
    df_class_0 = trainer[trainer[q] == 0]
    df_class_1 = trainer[trainer[q] == 1]
    if count_class_0>count_class_1:
        df_class_1_over = df_class_1.sample(count_class_0, replace=True)
        df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)
    else:
        df_class_0_over = df_class_0.sample(count_class_1,replace=True)
        df_test_over = pd.concat([df_class_0_over, df_class_1], axis=0)

    print('Random over-sampling:')
    print(df_test_over[q].value_counts())
    X = df_test_over.drop(q,axis='columns')
    y = df_test_over[q]

    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
    # loss = keras.losses.BinaryCrossentropy()
    # weights = -1
    y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)
    

In [59]:
# q = "Benazepril hydrochloride tablet"

In [60]:
# PCA_df = pd.concat([PCA_df, df[q]], axis = 1)

In [61]:
# PCA_df

### Run the Pipeline

In [62]:
for q in df.columns:
    print("Running for :",q)
    temp = pd.concat([PCA_df, df[q]], axis = 1)
    model(q,temp)


Running for : Atorvastatin calcium tablet
Random over-sampling:
0    1185
1    1185
Name: Atorvastatin calcium tablet, dtype: int64
Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.74      0.78       237
           1       0.76      0.84      0.80       237

    accuracy                           0.79       474
   macro avg       0.79      0.79      0.79       474
weighted avg       0.79      0.79      0.79       474

Running for : Benazepril hydrochloride tablet
Random over-sampling:
0    1569
1    1569
Name: Benazepril hydrochloride tablet, dtype: int64
Classification Report: 
               precision    recall  f1-score   support

           0       0.96      0.81      0.88       314
           1       0.83      0.96      0.89       314

    accuracy                           0.89       628
   macro avg       0.90      0.89      0.88       628
weighted avg       0.90      0.89      0.88       628

Running for : Deslanosid